In [7]:
import twitter
import datetime
import re
import time
import os
from socket import error as SocketError
import tweepy
import nltk, pprint, string, csv
from operator import itemgetter
import itertools
import ast, pickle, random  
from sklearn import svm
import numpy as np

if not os.path.isdir("Data/Testing_data/"):
    os.makedirs("Data/Testing_data/")

def tweets_collection(curr_pair, start_date, end_date):
    curr_pair_ = "'" + curr_pair + "'"
    consumer_key = "WkmvAytt1DEWva2VukcqACVtK"
    consumer_secret = "jy8D7yESo1KrUc0EgIXwVWw4IUHvSy2AgQzsMIXMUb6UtM0S9p"
    access_token = "447676855-FbQsPAuLttxllF8TB4eK6CV8keYZk7BEW6UCQDuw"
    access_secret = "NaFfyZMKHCP9zCpm88PcoGvOh7ZQNP3kUlPaxUjS44Vxz"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)

    def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)
    
    pairs = ['Pairs', curr_pair_]
        
    #read control_maximum file and get maximum id to use as a filter
    try:
        control = open("Data/Testing_data/maximum_id_"+curr_pair+".csv", "r")
        maxid = control.readline()
        control.close()
    except Exception as fileEx:
        maxid="0"
        fileToWriteError=open("Errors.txt","a")
        fileToWriteError.write("maxid: " + str(fileEx) +","+datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")+ "\n")

    twitter_search = twitter.Twitter(domain='api.twitter.com', api_version='1.1',  
                                     auth=twitter.oauth.OAuth('447676855-FbQsPAuLttxllF8TB4eK6CV8keYZk7BEW6UCQDuw', 
                                                              'NaFfyZMKHCP9zCpm88PcoGvOh7ZQNP3kUlPaxUjS44Vxz', 
                                                              'WkmvAytt1DEWva2VukcqACVtK', 
                                                              'jy8D7yESo1KrUc0EgIXwVWw4IUHvSy2AgQzsMIXMUb6UtM0S9p'))

    #hold the query results
    tweets_ids = []
    tweets = []
    timestamp = []
    tweets_users = []

    #run twitter search api
    try:    
        for tweet in tweepy.Cursor(api.search,
                               q=curr_pair_,
                               since=start_date,
                               until=end_date,
                               count = 100,
                               lang="en").items():
            tweets_ids.append(tweet.id), 
            tweets.append(tweet.text),
            timestamp.append(str(tweet.created_at)),
            tweets_users.append(tweet.user.name)

    except ValueError:
        fileToWriteErrorConnection=open("Errors.txt","a")
        fileToWriteErrorConnection.write("twitter_res append: " + str(ValueError) +","+datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")+ "\n")

    pairs = ['Pairs', curr_pair]

    for counter in range(0, len(tweets)):
        #remove hashtags, user tags and user retweets
        tweets[counter]=re.sub(r'(?<=#)\w+| #|(?<=@)\w+| @ |(?<=RT)\w+|RT',"", tweets[counter])
        #convert tweet to lowecase 
        tweets[counter] = tweets[counter].lower()
        tweets[counter]= re.sub(r'[^\w:.$]'," ",tweets[counter])
        #remove links
        tweets[counter] = re.sub(r'http\\w+',"",tweets[counter])
        tweets[counter]= re.sub(r'http:.{15}',"",tweets[counter])
        tweets[counter]= re.sub(r'http:.{10}',"",tweets[counter])

        for pairCounter in range(1, len(pairs)):
            currentPair = pairs[pairCounter].replace('/',"").replace('\r\n',"").lower()
            if currentPair in tweets[counter]:
                if not os.path.isdir("Data/Testing_data/"+currentPair.upper()):
                    os.makedirs("Data/Testing_data/"+currentPair.upper())
                try:
                    fileToWrite = open("Data/Testing_data/"+currentPair.upper()+"/"+currentPair.upper()+"TemporaryRaw"+str(start_date)+'.csv',"a")
                except Exception as fileEx:
                    fileToWriteError=open("Errors.txt","a")
                    fileToWriteError.write("tweetWrite: " + str(fileEx) +","+datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")+ "\n")
                fileToWrite = open("Data/Testing_data/"+currentPair.upper()+"/"+currentPair.upper()+"TemporaryRaw"+str(start_date)+'.csv',"a")
                tweets_users[counter] = removeNonAscii(tweets_users[counter])
                tweets[counter] = removeNonAscii(tweets[counter])
                fileToWrite.write(str(tweets_ids[counter]) +","+tweets_users[counter]+","
                                      +tweets[counter]+","+time.strftime(timestamp[counter])+"\n")
                fileToWrite.close()

    print ('Collected: ' + str(len(tweets_ids)) + ' tweet(s)')
            
    if len(tweets_ids) > 0:
        controlIdFile = open("Data/Testing_data/maximum_id_"+curr_pair+".csv","w")
        try:
            controlIdFile.write(str(max(tweets_ids)))
            controlIdFile.close()
        except Exception as fileEx:
            fileToWriteError=open("Errors.txt","a")
            fileToWriteError.write("maxID writeback: " + str(fileEx) +","+datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")+ "\n")

In [8]:
from datetime import date, timedelta, datetime
import datetime
import os.path
from tqdm import tqdm, tnrange

curr_list = ['EURUSD', 'GBPUSD', 'USDJPY', 'USDCAD', 'USDCHF']

# First day in database
date0 = date(2019, 1, 7) # year, month, day

# Today
today = datetime.date.today()

# Open txt file to write
text_file = open("Data/Database_content.txt", "r")
lines = text_file.readlines()
print(lines)
text_file.close()

for i in tnrange(150, desc='Tweet collection'):
    lower_bound = date0 + timedelta(days=i)
    upper_bound = lower_bound + timedelta(days=1)
    
    if today > lower_bound:
        if str(lower_bound) not in str(lines):
#             print('No such date --> work further to get it')
            text_file = open("Data/Database_content.txt", "a")
            start_date = lower_bound
            end_date = upper_bound
            
            text_file.write(str(lower_bound))
            text_file.write(' ')
            for c in curr_list:
                print('Run tweets collection for {} {}'.format(start_date, c))
                curr_pair = c
#                 print(curr_pair, start_date, end_date)
                tweets_collection(curr_pair, start_date, end_date)
text_file.close()

['2019-01-07 2019-01-08 2019-01-09 2019-01-11 2019-01-12 2019-01-13 2019-01-14 2019-01-10 2019-01-15 2019-01-16 2019-01-17 2019-01-18 2019-01-19 2019-01-20 2019-01-21 2019-01-22 2019-01-23 2019-01-24 2019-01-25 2019-01-26 2019-01-27 2019-01-28 2019-01-29 2019-01-30 2019-01-31 2019-02-01 2019-02-02 2019-02-03 2019-02-04 2019-02-05 2019-02-06 2019-02-07 2019-02-08 2019-02-09 2019-02-10 2019-02-11 2019-02-12 2019-02-13 2019-02-14 2019-02-15 2019-02-16 2019-02-17 2019-02-18 2019-02-19 2019-02-20 2019-02-21 2019-02-22 2019-02-23 2019-02-24 2019-02-25 2019-02-26 2019-02-27 2019-02-28 2019-03-01 2019-03-02 2019-03-03 2019-03-04 2019-03-05 2019-03-06 2019-03-07 2019-03-08 2019-03-09 2019-03-10 2019-03-11 2019-03-12 2019-03-13 2019-03-14 2019-03-15 2019-03-16 2019-03-17 2019-03-18 2019-03-19 2019-03-20 2019-03-21 2019-03-22 2019-03-23 2019-03-24 2019-03-25 2019-03-26 2019-03-27 2019-03-28 2019-03-29 2019-03-30 2019-03-31 2019-04-01 2019-04-02 2019-04-03 2019-04-04 2019-04-05 2019-04-06 2019-04-

Run tweets collection for 2019-05-27 EURUSD
Collected: 1068 tweet(s)
Run tweets collection for 2019-05-27 GBPUSD
Collected: 764 tweet(s)
Run tweets collection for 2019-05-27 USDJPY
Collected: 478 tweet(s)
Run tweets collection for 2019-05-27 USDCAD
Collected: 318 tweet(s)
Run tweets collection for 2019-05-27 USDCHF
Collected: 245 tweet(s)
